# Deploy a Serverless Model Server with Nuclio-KFServing
  --------------------------------------------------------------------

The following notebook demonstrates how to deploy **any pickled model** using **[nuclio](https://github.com/nuclio/nuclio)** + **[KFServing](https://github.com/kubeflow/kfserving)** (a.k.a <b>Nuclio-serving</b>)

#### **notebook how-to's**
* Write and test model serving (KFServing) class in a notebook.
* Deploy the model server as a Nuclio-serving function.
* Invoke and test the serving function.

<a id='top'></a>
#### **steps**
**[define a new function and its dependencies](#define-function)**<br>
**[test the model serving class locally](#test-locally)**<br>
**[deploy our serving class using as a serverless function](#deploy)**<br>
**[test our model server using HTTP request](#test-model-server)**<br>

In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio

<a id='define-function'></a>
### **define a new function and its dependencies**

In [2]:
# %nuclio config spec.build.baseImage = "yjbds/mlrun-serving:dev"

In [3]:
%%nuclio cmd -c
python -m pip install -U -q kfserving
python -m pip install -U nuclio-sdk==0.0.7
python -m pip install -U lightgbm xgboost

python -m pip install -U -q git+https://github.com/mlrun/mlrun.git@development

In [4]:
# %nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=ClassifierModel

%nuclio: setting 'MODEL_CLASS' environment variable


In [5]:
import kfserving
import os
import numpy as np
from cloudpickle import load

In [6]:
TARGET_PATH = '/User/repos/demos/dask/artifacts'
MODEL_FILE = 'xgb-model.pkl'

In [7]:
class ClassifierModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str, model = None):
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if not model is None:
            self.classifier = model
            self.ready = True

    def load(self):
        model_file = os.path.join(
            kfserving.Storage.download(self.model_dir), MODEL_FILE)
        self.classifier = load(open(model_file, 'rb'))
        self.ready = True

    def predict(self, body):
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

The following end-code annotation tells ```nuclio``` to stop parsing the notebook from this cell. _**Please do not remove this cell**_:

In [8]:
# nuclio: end-code

______________________________________________

<a id='test-locally'></a>
### **test the model serving class locally**
The class above can be tested locally. Just instantiate the class, `.load()` will load the model to a local dir.

In [9]:
# model = load(open(TARGET_PATH + '/' + MODEL_FILE, 'rb'))

# my_server = ClassifierModel('classifier', model_dir=TARGET_PATH, model = model)
# my_server.load()

### _data_
Make some classification data using scikit learn's `make_classification`:

In [10]:
import pandas as pd

xtest = pd.read_parquet('/User/repos/demos/dask/dataset/partitions')

In [11]:
xtest = xtest.sample(100)
ytest = xtest.pop('ArrDelay')

In [12]:
event = {"instances": xtest.values.tolist()}
# my_server.predict(event)

<a id='deploy'></a>
### **deploy our serving class using as a serverless function**

In [13]:
import mlrun

In [14]:
fn = mlrun.new_model_server('generic', 
                            models={'classifier_gen': TARGET_PATH}, 
                            model_class='ClassifierModel')
fn.apply(mlrun.mount_v3io())

In [15]:
# %nuclio show

In [16]:
# print(fn.to_yaml())

In [17]:
fn.spec.no_cache = True
addr = fn.deploy()

[mlrun] 2020-02-19 09:22:06,445 deploy started
[nuclio] 2020-02-19 09:22:07,529 (info) Building processor image
[nuclio] 2020-02-19 09:23:44,432 (info) Build complete
[nuclio] 2020-02-19 09:23:50,499 (info) Function deploy complete
[nuclio] 2020-02-19 09:23:50,504 done updating generic, function address: 3.12.228.39:30130


<a id="test-model-server"></a>
### **test our model server using HTTP request**

In [21]:
import json
import requests

resp = requests.post(addr + '/classifier_gen/predict', json=event)

In [22]:
resp.__dict__['_content'] 

b'Exception caught in handler "Failed to predict could not convert string to float: \'DL\'": Traceback (most recent call last):\n  File "/opt/nuclio/model_server.py", line 29, in predict\n    result: np.ndarray = self.classifier.predict(feats)\n  File "/usr/local/lib/python3.6/site-packages/lightgbm/sklearn.py", line 814, in predict\n    pred_leaf, pred_contrib, **kwargs)\n  File "/usr/local/lib/python3.6/site-packages/lightgbm/sklearn.py", line 863, in predict_proba\n    pred_leaf, pred_contrib, **kwargs)\n  File "/usr/local/lib/python3.6/site-packages/lightgbm/sklearn.py", line 665, in predict\n    pred_leaf=pred_leaf, pred_contrib=pred_contrib, **kwargs)\n  File "/usr/local/lib/python3.6/site-packages/lightgbm/basic.py", line 2415, in predict\n    data_has_header, is_reshape)\n  File "/usr/local/lib/python3.6/site-packages/lightgbm/basic.py", line 535, in predict\n    preds, nrow = self.__pred_for_np2d(data, num_iteration, predict_type)\n  File "/usr/local/lib/python3.6/site-package

In [20]:
json.loads(resp.content)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

**[back to top](#top)**